In [1]:
import asyncio
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker
from sqlalchemy.future import select
from statsmodels.api import OLS
import pandas as pd
from sqlalchemy import case, func
from fleecekmbackend.db.models import Author, Answer, Rating
from fleecekmbackend.core.config import DATABASE_URL
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Setting up the async engine and session
engine = create_async_engine(DATABASE_URL, echo=False)
async_session = sessionmaker(engine, expire_on_commit=False, class_=AsyncSession)

async def fetch_data(model_name):
    async with async_session() as session:
        # Define a subquery for authors using the specific model
        author_subquery = select(Author.id).where(Author.model == model_name).subquery()

        # Define a subquery for questions having both 'ic' and 'zs' answers from the same author
        answer_subquery = select(Answer.question_id).join(
            author_subquery, Answer.author_id == author_subquery.c.id
        ).group_by(
            Answer.question_id, Answer.author_id
        ).having(
            func.count(Answer.id) >= 2,  # Assuming at least one 'ic' and one 'zs'
            func.sum(case((Answer.setting == 'ic', 1), else_=0)) > 0,
            func.sum(case((Answer.setting == 'zs', 1), else_=0)) > 0
        ).subquery()

        # Main query to select answers and their ratings
        results = await session.execute(
            select(Answer.question_id, Answer.setting, Rating.value)
            .join(Rating, Answer.id == Rating.answer_id)
            .join(answer_subquery, Answer.question_id == answer_subquery.c.question_id)
            .select_from(Answer)  # Explicitly start from Answer table
            .limit(100)  # Adjust as needed based on expected pairs
        )
        return results.all()

def prepare_data(data):
    # Create a DataFrame
    df = pd.DataFrame([{
        'question_id': question_id,
        'setting': setting,
        'rating_value': value
    } for question_id, setting, value in data])
    
    # Convert 'setting' to a binary indicator variable for regression analysis
    df['ic'] = (df['setting'] == 'ic').astype(int)
    return df

async def analyze_model(model_name):
    data = await fetch_data(model_name)
    prepared_data = prepare_data(data)
    
    # Perform regression analysis
    Y = prepared_data['rating_value']
    X = prepared_data[['ic']]
    X = sm.add_constant(X)  # adding a constant
    model = OLS(Y, X).fit()
    print(f"Regression Analysis for {model_name}")
    print(model.summary())
    
    # Plotting the data
    prepared_data.groupby(['setting'])['rating_value'].mean().plot(kind='bar')
    plt.title(f'Average Rating by Setting for {model_name}')
    plt.xlabel('Setting')
    plt.ylabel('Average Rating')
    plt.show()


In [7]:
async def test_author_subquery(model_name):
    async with async_session() as session:
        author_subquery = select(Author.id, Author.model, Author.prompt).where(Author.model == model_name).subquery()
        result = await session.execute(select(author_subquery.c.id, author_subquery.c.prompt, author_subquery.c.model).limit(10))
        print(result.all())

await test_author_subquery('gpt-4-turbo')

[(9, '<str>Answer the following question in a succinct manner: <str>', 'gpt-4-turbo')]


In [9]:
async def fetch_answers_by_model(model_name):
    async with async_session() as session:
        # Define the subquery to fetch author IDs based on the model
        author_subquery = select(Author.id).where(Author.model == model_name).subquery()

        # Main query to fetch answers based on the author IDs from the subquery
        results = await session.execute(
            select(Answer.id, Answer.text, Answer.question_id, Answer.author_id)
            .join(author_subquery, Answer.author_id == author_subquery.c.id)
        )
        return results.all()

# Calling the function with the model name 'gpt-4-turbo'
answers = await fetch_answers_by_model('gpt-4-turbo')
answers

[(976237, 'Kevin Kelley was the drummer for The Byrds in 1968.', 22086, 9),
 (976239, 'In 1968, the drummer for The Byrds was Michael Clarke.', 22086, 9),
 (976301, 'The West Wing won nine Emmy Awards during its debut season.', 4178, 9),
 (976304, 'The West Wing won 9 Emmy Awards during its debut season.', 4178, 9),
 (976492, 'Paraguay', 184843, 9),
 (976496, 'The Fortress of Humaitá was built to increase the security of Paraguay.', 184843, 9),
 (976515, 'To gain membership at many gay bathhouses, customers generally need to pay a small fee and membership is typically open to any adult who seeks it.', 55883, 9),
 (976520, "To gain membership at many gay bathhouses, customers typically need to provide a valid ID and pay a membership fee. Some bathhouses may also require agreeing to the establishment's rules and policies.", 55883, 9),
 (976543, 'The 66th Venice International Film Festival.', 246790, 9),
 (976548, 'The film "Mr. Nobody" had its world premiere at the Venice Film Festival o

In [13]:
async def test_answer_subquery(model_name):
    async with async_session() as session:
        author_subquery = select(Author.id).where(Author.model == model_name).subquery()
        answer_subquery = select(Answer.question_id).join(
            author_subquery, Answer.author_id == author_subquery.c.id
        ).group_by(
            Answer.question_id, Answer.author_id
        ).having(
            func.count(Answer.id) >= 2,
            func.sum(case((Answer.setting == 'ic', 1), else_=0)) > 0,
            func.sum(case((Answer.setting == 'zs', 1), else_=0)) > 0
        ).subquery()
        result = await session.execute(select(answer_subquery.c.question_id))
        return result.all()

res = await test_answer_subquery('gpt-4-turbo')
print(len(res))

62


In [14]:
data = await fetch_data('gpt-4-turbo')
data

[]

In [ ]:
prepared_data = prepare_data(data)
prepared_data

In [ ]:
await analyze_model('gpt-4-turbo')

In [16]:
from sqlalchemy import func

async def get_setting_counts():
    async with async_session() as session:
        # Query to count each type of setting
        results = await session.execute(
            select(Answer.setting, func.count(Answer.setting))
            .group_by(Answer.setting)
        )
        return results.all()

# Example of how to run the function in an asyncio environment
# Since we are in Jupyter, you can directly run this with `await` in a cell
counts = await get_setting_counts()
print(counts)

[('ic', 859725), ('human', 36), ('zs', 1736)]


In [ ]:

async def perform_regression(df):
    # Performing regression
    X = df[['setting']]  # Independent variable
    y = df['rating_value']  # Dependent variable
    model = OLS(y, X).fit()
    return model.summary()